In [ ]:
import h5py
import numpy as np
from matplotlib import pyplot as plt
import scipy.ndimage
import yaml
from pyevtk.hl import imageToVTK 
from skimage import io
from scipy.ndimage import gaussian_filter
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops

# read parameters

In [ ]:
with open("../base/parameters.yml") as file:
    parameters = yaml.load(file, Loader=yaml.FullLoader)
with open("../local/parameters.yml") as file:
    parameters_local = yaml.load(file, Loader=yaml.FullLoader)

# overwrite global parameters with local setting
for key in parameters_local:
    parameters[key] = parameters_local[key]
    
print(parameters)

# specify and read tiff images

In [ ]:
data_folder = parameters["data_folder"]

filename_short = "200811_Pos002"
macrophage_file = "200811 mpeg-mCherry x kdrl-lynEYFP LBT123 1in27 GFP 2dpf 1dpi Ti30min_Mark_and_Find003_Position002_macrophages3D.tif"
im_macrophages = np.array(io.imread(data_folder + macrophage_file))
tumor_file = "200811 mpeg-mCherry x kdrl-lynEYFP LBT123 1in27 GFP 2dpf 1dpi Ti30min_Mark_and_Find003_Position002_tumor3D.tif"
im_tumor = np.array(io.imread(data_folder + tumor_file))
vessel_file = "200811 mpeg-mCherry x kdrl-lynEYFP LBT123 1in27 GFP 2dpf 1dpi Ti30min_Mark_and_Find003_Position002_vessels3D.tif"
im_vessel = np.array(io.imread(data_folder + vessel_file))
im_vessel.shape

# smooth images

In [ ]:
sigma0 = 5
im_macrophage_smoothed = gaussian_filter(im_macrophages[0,:,:,:], sigma = sigma0)
im_tumor_smoothed = gaussian_filter(im_tumor[0,:,:,:], sigma = sigma0)
im_vessel_smoothed = gaussian_filter(im_vessel[0,:,:,:], sigma = sigma0)

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
ax.imshow(im_macrophage_smoothed[100,:,:])

# save time series

In [ ]:
sigma0 = 5
output_folder = parameters["output_folder"]

for frame in range(0,im_vessel.shape[0]):
    print(str(frame).zfill(3))    
    
    im_macrophages_smoothed = gaussian_filter(im_macrophages[frame,:,:,:], sigma = sigma0)
    im_tumor_smoothed = gaussian_filter(im_tumor[frame,:,:,:], sigma = sigma0)    
    im_vessel_smoothed = gaussian_filter(im_vessel[frame,:,:,:], sigma = sigma0)    
        
    time_stamp = str(frame).zfill(3)
    filename = output_folder + filename_short + time_stamp
    imageToVTK(filename, cellData = {"macrophages" : im_macrophages_smoothed, 
                                     "tumor": im_tumor_smoothed, 
                                     "vessel": im_vessel_smoothed})